## Blending
Blending is the merging of two different images into one. It can be horizontal blending or vertical blending. But, blending, generally is the gradual blending and there isn't any sharp line of distinction between the two images. 

At first, we will try simple blending, where there is a sharp line of distinction between the two joined images.

In [2]:
import cv2
import numpy as np

In [3]:
apple = cv2.imread('data/data_cv2/apple.jpg')
orange = cv2.imread('data/data_cv2/orange.jpg')

In [4]:
print(apple.shape)
print(orange.shape)

(512, 512, 3)
(512, 512, 3)


In [5]:
#Blending the left part of apple with right part of orange using hstack()
apple_orange = np.hstack((apple[:,:256], orange[:,256:]))
cv2.imshow('apple', apple)
cv2.imshow('Orange', orange)
cv2.imshow('apple_orange', apple_orange)
cv2.waitKey(0)
cv2.destroyAllWindows()

Using the above piece of code we see that the images are just joined but not perfectly blended. That is, we can see a sharp line of demarcation between the two images. 

Now we will try to blend the images using **image pyramids technique**. 

The steps involved are as follows:
1. Load the two images to be blended.
2. Find the **Gaussian Pyramids** for the 2 images (apple and orange here, the number of levels here is 6).
3. From Gaussain Pyramids find their Laplacian Pyramids. 
4. Now join the images at each individual level of the pyramids. 
5. Finally, from this joint image pyramids, reconstruct the original image. 

In [9]:
# Generating Gaussian Pyramid for apple:
apple_copy = apple.copy()
gpyramid_apple = [apple_copy]
for i in range(6):
    apple_copy = cv2.pyrDown(apple_copy)
    gpyramid_apple.append(apple_copy)

    
# Generating Gaussian Pyramid for orange:
orange_copy = orange.copy()
gpyramid_orange = [orange_copy]
for i in range(6):
    orange_copy = cv2.pyrDown(orange_copy)
    gpyramid_orange.append(orange_copy)
    
    
# Generating Laplacian Pyramid for apple:
apple_copy = gpyramid_apple[5]
lpyramid_apple = [apple_copy]
for i in range(5,0,-1):
    gaussian_expanded_apple = cv2.pyrUp(gpyramid_apple[i])
    laplacian = cv2.subtract(gpyramid_apple[i-1], gaussian_expanded_apple)
    lpyramid_apple.append(laplacian)
    
    
# Generating Laplacian Pyramid for orange:
orange_copy = gpyramid_orange[5]
lpyramid_orange = [orange_copy]
for i in range(5,0,-1):
    gaussian_expanded_orange = cv2.pyrUp(gpyramid_orange[i])
    laplacian = cv2.subtract(gpyramid_orange[i-1], gaussian_expanded_orange)
    lpyramid_orange.append(laplacian)
    
    
# Now joining the halves of these two images in each level
apple_orange_pyramid = []
n = 0
for apple_lap, orange_lap in zip(lpyramid_apple, lpyramid_orange):
    n = n + 1
    columns, rows, channels = apple_lap.shape
    joint_laplacian = np.hstack((apple_lap[:,:int(columns/2)], orange_lap[:,int(columns/2):]))
    apple_orange_pyramid.append(joint_laplacian)
    
    
# Now reconstructing the original image
apple_orange_reconstructed = apple_orange_pyramid[0]
for i in range(1,6):
    apple_orange_reconstructed = cv2.pyrUp(apple_orange_reconstructed)
    apple_orange_reconstructed = cv2.add(apple_orange_pyramid[i], apple_orange_reconstructed)
    

cv2.imshow('apple', apple)
cv2.imshow('Orange', orange)
cv2.imshow('apple_orange', apple_orange)
cv2.imshow('Apple orange blend', apple_orange_reconstructed)
cv2.waitKey(0)
cv2.destroyAllWindows()